In [ ]:
#
# use array visited
#
site_url = "https://arthew0.ru/"
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import random
import tldextract

urls = [site_url]
visited = []
def clean_url(url):
    return url.replace("www.", "")

while len(urls) != 0:
    random.shuffle(urls)
    current_url = urls.pop()
    visited.append(current_url)
    current_domain = urlparse(current_url).hostname
    current_site = urlparse(current_url).scheme + "://" + urlparse(current_url).netloc
    try:
        response = requests.get(current_url, timeout=1)
        soup = BeautifulSoup(response.content, "html.parser", from_encoding="utf-8")
        print(len(urls), len(visited), current_domain, current_url)
        link_elements = soup.select("a[href]")
        for link_element in link_elements:
            url = link_element['href']
            url = requests.compat.urljoin(current_site, url)
            url = clean_url(url)
            if not url in visited and not "javascript" in url:
                urls.append(url)
        if len(visited) > 1000:
            break

    except Exception as e:
        pass



In [ ]:
#
# database check
#
create = False
fill = False
db_name = "d1b.db"

def get_values(url):
    return {"hostname": urlparse(url).hostname, "url": url, "visited":0}

if create:
    import os
    os.remove(db_name)

from databases import Database
database = Database(f'sqlite+aiosqlite:///{db_name}')

if create:
    await database.connect()
    query = """CREATE TABLE Urls (id INTEGER PRIMARY KEY, hostname VARCHAR(127), url VARCHAR(127), visited INTEGER)"""
    await database.execute(query=query)
    query = """CREATE TABLE Domains (id INTEGER PRIMARY KEY AUTOINCREMENT, hostname VARCHAR(127), visited INTEGER)"""
    await database.execute(query=query)

if fill:
    for i in range(0, 3):
        values = get_values(f"https://arthew0.ru/page{i}")
        values['visited'] = i % 2
        query = "INSERT INTO Urls(hostname, url, visited) VALUES (:hostname, :url, :visited)"
        await database.execute(query=query, values=values)
    for i in range(0, 15):
        values = get_values(f"https://ya.ru/page{i}")
        values['visited'] = i % 2
        query = "INSERT INTO Urls(hostname, url, visited) VALUES (:hostname, :url, :visited)"
        await database.execute(query=query, values=values)

# next url:
query = "SELECT id, hostname, url, count(visited) FROM Urls where visited==0 GROUP BY hostname ORDER BY count(visited) LIMIT 1"
rows = await database.fetch_all(query=query)
url_id = rows[0][0]
next_url = rows[0][2]
print('next_url:', url_id, next_url)

query = f"UPDATE Urls SET visited=1 WHERE id={url_id}"
await database.execute(query=query)

#query = "SELECT * FROM Urls"
#rows = await database.fetch_all(query=query)
#print('urls:', rows)


In [ ]:
db_name = "db.db"
from databases import Database
database = Database(f'sqlite+aiosqlite:///{db_name}')
await database.connect()
query = "SELECT * FROM Urls LIMIT 1000,2000"
rows = await database.fetch_all(query=query)
# print('urls:', rows)

display(rows)

# work version

In [ ]:
def clean_url(url):
    return url.replace("www.", "")

In [ ]:
import csv
filters = []
with open('top500Domains.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='\"')
    for row in spamreader:
        filters.append(clean_url(''.join(row[1])))


In [ ]:

import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import random
import tldextract
from datetime import date
from tld import get_tld

#
# use database visited
#
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%Y-%m-%d_%H-%M")

create = True
fill = False
db_name = f"db_{date_time}.db"

def get_values(url):
    return {"hostname": urlparse(url).hostname, "url": url, "visited":0}

if create:
    import os
    if os.path.exists(db_name):
        os.remove(db_name)

from databases import Database
database = Database(f'sqlite+aiosqlite:///{db_name}')
await database.connect()

if create:
    query = """CREATE TABLE Urls (id INTEGER PRIMARY KEY, hostname VARCHAR(127), url VARCHAR(127) unique, src_url VARCHAR(127), visited INTEGER)"""
    await database.execute(query=query)

values = get_values(f"https://arthew0.ru/")
query = "INSERT INTO Urls(hostname, url, visited) VALUES (:hostname, :url, :visited)"
await database.execute(query=query, values=values)


#site_url = "https://arthew0.ru/"
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import random
import tldextract

step = 0
while True:
    step = step + 1
    query = "SELECT id, hostname, url, src_url, count(visited) FROM Urls where visited==0 GROUP BY hostname ORDER BY count(visited) LIMIT 1"
    rows = await database.fetch_all(query=query)
    try:
        url_id = rows[0][0]
        current_url = rows[0][2]
        src_url = rows[0][3]
        query = f"UPDATE Urls SET visited=1 WHERE id={url_id}"
        await database.execute(query=query)
        current_domain = urlparse(current_url).hostname
        current_site = urlparse(current_url).scheme + "://" + urlparse(current_url).netloc
        res = get_tld(current_url, as_object=True)
        current_base_domain = res.fld
        try:
            response = requests.get(current_url, timeout=1)
            soup = BeautifulSoup(response.content, "html.parser", from_encoding="utf-8")
            print(step, src_url, ">" ,current_url)
            link_elements = soup.select("a[href]")
            for link_element in link_elements:
                url = link_element['href']
                url = requests.compat.urljoin(current_site, url)
                url = clean_url(url)
                if not "javascript" in url and not current_base_domain in filters:
                    values = get_values(url)
                    values['src_url'] = current_url
                    query = "INSERT OR IGNORE INTO Urls(hostname, url, src_url, visited) VALUES (:hostname, :url, :src_url, :visited)"
                    await database.execute(query=query, values=values)
                if step > 50:
                    break

        except Exception as e:
            print("Exception1:", e)
            pass

    except Exception as e:
        print("Exception2:", e)
        pass

database.disconnect()

In [ ]:
database.disconnect()

# Custom class